<a href="https://colab.research.google.com/github/deepakproai/Fake_news_Machine_Hack/blob/master/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn.metrics import *

!pip install catboost
from catboost import CatBoostClassifier

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 66.1MB 42kB/s 


In [ ]:
pd.set_option('colwidth',4000)

In [ ]:
train = pd.read_csv(r"/content/sample_data/Train.csv")

In [ ]:
train.head()

,Labels,Text,Text_Tag
0,1,Says the Annies List political group supports third-trimester abortions on demand.,abortion
1,2,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments"
2,3,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy
3,1,Health care reform legislation is likely to mandate free sex change surgeries.,health-care
4,2,The economic turnaround started at the end of my term.,"economy,jobs"


In [ ]:
train['Labels'].value_counts()

2    2114
1    1995
3    1962
5    1676
0    1654
4     839
Name: Labels, dtype: int64

In [ ]:
train['Text_Tag'].value_counts()

health-care                                  381
taxes                                        308
immigration                                  253
elections                                    252
education                                    237
                                            ... 
history,population                             1
children,education,legal-issues,sexuality      1
gays-and-lesbians,military,abc-news-week       1
debates,small-business,taxes                   1
afghanistan,crime,immigration                  1
Name: Text_Tag, Length: 3827, dtype: int64

In [ ]:
train['Text_Len'] = train['Text'].apply(lambda x:len(x.split(' ')))

In [ ]:
train.head()

,Labels,Text,Text_Tag,Text_Len
0,1,Says the Annies List political group supports third-trimester abortions on demand.,abortion,11
1,2,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",24
2,3,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy,19
3,1,Health care reform legislation is likely to mandate free sex change surgeries.,health-care,12
4,2,The economic turnaround started at the end of my term.,"economy,jobs",10


In [ ]:
train.shape

(10240, 4)

In [ ]:
train.isnull().sum()

Labels      0
Text        0
Text_Tag    2
Text_Len    0
dtype: int64

In [ ]:
train[train['Text_Tag'].isna()]

,Labels,Text,Text_Tag,Text_Len
2142,1,"The fact is that although we have had a president who is opposed to abortion over the last eight years, abortions have not gone down.''\tabortion\tbarack-obama\tPresident\tIllinois\tdemocrat\t70\t71\t160\t163\t9\ta TV interview with megachurch pastor Rick Warren in Lake Forest, Calif.\n2724.json\ttrue\tMost of the jobs that we lost were lost before the economic policies we put in place had any effect.\teconomy,job-accomplishments,jobs,stimulus\tbarack-obama\tPresident\tIllinois\tdemocrat\t70\t71\t160\t163\t9\tan interview on The Daily Show with Jon Stewart""",NaN,63
9375,1,"Joe, I keep hearing you every morning talking about the biggest tax increase in history, but you don't mention it's also the biggest tax cut in history.''\thealth-care,taxes\trichard-durbin\tSenator\tIllinois\tdemocrat\t0\t2\t1\t0\t1\ta comment on the Morning Joe'' show on MSNBC.",NaN,35


Attribute Description:

    Text - Raw content from social media/ new platforms
    Text_Tag - Different types of content tags
    Labels - Represents various classes of Labels
        Half-True - 2
        False - 1
        Mostly-True - 3
        True - 5
        Barely-True - 0
        Not-Known - 4

In [ ]:


import nltk
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer



     |████████████████████████████████| 71kB 2.5MB/s 
     |████████████████████████████████| 890kB 5.2MB/s 
     |████████████████████████████████| 890kB 10.0MB/s 
     |████████████████████████████████| 1.1MB 24.7MB/s 
     |████████████████████████████████| 3.0MB 23.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.6-cp36-none-any.whl size=101182 sha256=bc85e9700ab382f45e8e6d2fe272cc0ba188940b3504af1e36a41d46d91aaff7
  Stored in directory: /root/.cache/pip/wheels/6f/3f/75/c0c4b3ef5dfbf8806d37b8dc661861772aba2f7aa419c85a9b
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d160b3575bc4bc5ec9aeb4bc305f70964ef0d2cbc1615b615ee69ce2ea3cf04e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
test = pd.read_csv(r"/content/sample_data/Test.csv")

In [ ]:
test.head()

,Text,Text_Tag
0,Building a wall on the U.S.-Mexico border will take literally years.,immigration
1,Wisconsin is on pace to double the number of layoffs this year.,jobs
2,Says John McCain has done nothing to help the vets.,"military,veterans,voting-record"
3,Suzanne Bonamici supports a plan that will cut choice for Medicare Advantage seniors.,"medicare,message-machine-2012,campaign-advertising"
4,"When asked by a reporter whether hes at the center of a criminal scheme to violate campaign laws, Gov. Scott Walker nodded yes.","campaign-finance,legal-issues,campaign-advertising"


#GENERATING WORD EMBEDDINGS AND A FEW NEW FEATURES

In [ ]:


full_df = pd.concat([train, test]).reset_index()



In [ ]:
word_embeddings = SentenceTransformer('roberta-large-nli-stsb-mean-tokens').encode(full_df.Text)

100%|██████████| 1.31G/1.31G [00:47<00:00, 27.5MB/s]


In [ ]:
word_embeddings = pd.DataFrame(data=word_embeddings)

In [ ]:
word_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-0.843113,0.354266,0.356839,0.419612,0.502846,0.265881,1.124245,-0.337539,-1.011597,0.389831,-1.143914,-1.498768,0.349013,0.630598,-0.308929,-1.211497,0.639380,-0.070070,0.197310,-0.694955,-0.748948,-0.985393,-1.381263,-0.965011,0.196261,-0.165667,-0.192807,-0.399925,-0.571577,-0.015549,0.400864,-0.732438,0.259238,0.038886,-1.680591,-0.752563,0.343458,-0.785571,-1.548424,-0.629427,...,-0.416050,-0.434246,-0.105925,0.207332,-0.963040,-0.041007,-0.155754,0.277735,0.030981,-0.246306,0.552735,-0.799284,-0.250765,-1.899152,-0.518160,0.620628,0.464231,-0.309856,0.085772,0.445845,0.001798,0.197886,1.256550,-0.412361,-0.562702,-1.153980,0.355673,-0.240783,-1.033721,1.400332,0.022141,0.510811,0.853409,0.351012,0.043098,0.612751,-0.362309,0.728115,-0.631633,-0.265538
1,-0.154687,0.872684,1.772497,-0.295066,0.222306,-0.253726,-1.300731,0.190504,0.343304,-0.872295,0.883556,-0.409843,-0.357488,-0.139657,0.384983,-0.794849,-1.066065,-0.051639,0.139327,0.139749,0.117898,0.461543,-1.046779,0.199070,-1.169691,-0.055779,-0.828227,-1.178741,0.154195,-1.155196,0.319821,-0.137026,-0.893601,-0.493398,0.772720,-0.778966,-0.642559,-0.128265,1.243242,-1.194343,...,0.877195,0.133658,-0.019471,-0.440079,0.622855,-0.382756,-0.269157,-0.771482,-0.626759,-1.214525,1.213236,-0.706025,-0.896191,-0.587146,0.188100,0.173347,-1.696494,-0.575991,-0.401825,-0.878120,0.333844,-0.645788,-0.933316,0.095632,-1.189360,-0.747389,-0.634917,1.291249,-1.595213,0.180152,-0.493622,0.729952,-0.513198,1.092363,-0.258281,0.267083,-0.399990,-0.463839,-1.466221,0.220743
2,-1.090439,-1.039243,-0.442930,-1.469323,0.165641,-0.411887,-0.524987,1.435755,-0.221388,0.336802,-0.551851,-0.579335,-1.103295,1.014058,0.514499,0.020389,1.099066,1.208101,0.988841,1.010967,-0.034147,-1.322206,-1.929260,-0.313306,-0.597575,0.154907,0.451024,-0.220749,-0.292568,-0.569620,-1.047028,-1.128314,0.382467,1.066755,-0.906427,-0.978917,0.257975,-0.024066,-1.304759,0.307851,...,0.383584,-0.643162,-0.567528,0.672795,0.460016,1.943318,-0.099922,-0.545651,0.080036,-0.051010,0.285562,0.600908,-0.681900,1.709387,-0.631731,0.168582,-0.115423,0.145797,-1.153039,0.255021,-1.061557,0.622468,0.076881,-1.268889,0.564817,0.068600,-0.259890,0.401971,-0.329564,1.528328,0.896938,1.188235,-0.614642,0.053428,-0.562576,0.325780,-1.103372,-1.954507,0.481241,-0.146407
3,0.599440,0.623602,0.633819,0.405363,0.706752,-0.079211,0.344765,-0.220469,-0.569685,-0.079065,-0.076758,1.569412,-0.278345,0.464309,-0.433065,-1.224822,-0.210659,-0.190351,-0.811026,-1.186909,-0.193128,0.878078,1.618985,-0.496415,1.310311,-0.229879,-0.483367,-0.503116,0.748487,-0.606732,-1.408139,-0.482079,-0.711370,0.238348,-0.895124,0.667424,-0.624264,-0.758625,-0.358398,-0.065462,...,-0.312904,-0.635221,-0.146240,-0.417504,-0.173062,0.587925,0.703627,0.999841,-0.951558,-0.135231,0.494266,-0.434182,-1.080807,-0.713855,-0.633628,0.060904,0.127755,-0.921143,-0.846210,-0.678195,-0.489322,-1.591884,0.925410,-0.127750,0.404707,-0.285969,0.945852,-0.417142,-1.571512,1.861141,0.311482,0.002135,-0.285264,0.073709,0.787635,-0.344154,-0.042826,1.363384,0.595176,-0.336081
4,-0.092456,-0.040649,-0.215230,0.439570,1.114174,0.060738,-0.115374,-0.424731,0.706268,-0.155484,0.662839,0.791876,-0.067190,-0.313515,-0.166140,0.178111,-0.376365,0.253623,-1.166783,-1.275674,0.633949,0.408042,-1.205060,-1.429486,-0.148765,0.656819,-1.473106,0.556760,-0.075808,-1.473760,0.632215,1.001486,0.441519,0.425402,-0.661800,2.639184,0.492242,0.714776,0.791692,-0.303791,...,0.680720,1.496218,-0.320826,0.126426,0.195708,0.626634,-0.573824,0.262326,-0.785340,-0.277521,2.571323,1.039394,-0.520890,-1.399412,1.457771,0.471627,-2.558429,-0.643707,-0.414140,0.643783,0.277404,-2.057265,-0.369222,0.34327

In [ ]:


from sklearn.decomposition import PCA
pca = pd.DataFrame(data=PCA(10).fit_transform(word_embeddings), columns=[f'pca{i}' for i in range(1,11)])



In [ ]:
full_df['Length'] = full_df['Text'].apply(len)

In [ ]:
full_df

,index,Labels,Text,Text_Tag,Text_Len,Length
0,0,1.0,Says the Annies List political group supports third-trimester abortions on demand.,abortion,11.0,82
1,1,2.0,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",24.0,141
2,2,3.0,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy,19.0,105
3,3,1.0,Health care reform legislation is likely to mandate free sex change surgeries.,health-care,12.0,78
4,4,2.0,The economic turnaround started at the end of my term.,"economy,jobs",10.0,54
...,...,...,...,...,...,...
11502,1262,NaN,Says his budget provides the highest state funding level in history for education.,education,NaN,82
11503,1263,NaN,Ive been here almost every day.,"civil-rights,crime,criminal-justice",NaN,31
11504,1264,NaN,"In the early 1980s, Sen. Edward Kennedy secretly offered to help Soviet leaders counter the Reagan administrations position on nuclear disarmament.","bipartisanship,congress,foreign-policy,history",NaN,147
11505,1265,NaN,Says an EPA permit languished under Strickland but his new EPA director got it done in two days.,"environment,government-efficiency",NaN,96


In [ ]:
full_df = full_df.drop(['Text_Len'],axis=1)

In [ ]:
full_df

,index,Labels,Text,Text_Tag,Length
0,0,1.0,Says the Annies List political group supports third-trimester abortions on demand.,abortion,82
1,1,2.0,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",141
2,2,3.0,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy,105
3,3,1.0,Health care reform legislation is likely to mandate free sex change surgeries.,health-care,78
4,4,2.0,The economic turnaround started at the end of my term.,"economy,jobs",54
...,...,...,...,...,...
11502,1262,NaN,Says his budget provides the highest state funding level in history for education.,education,82
11503,1263,NaN,Ive been here almost every day.,"civil-rights,crime,criminal-justice",31
11504,1264,NaN,"In the early 1980s, Sen. Edward Kennedy secretly offered to help Soviet leaders counter the Reagan administrations position on nuclear disarmament.","bipartisanship,congress,foreign-policy,history",147
11505,1265,NaN,Says an EPA permit languished under Strickland but his new EPA director got it done in two days.,"environment,government-efficiency",96


In [ ]:
full_df = pd.concat([full_df.drop('Text', axis=1), word_embeddings, pca], axis=1)
full_df



,index,Labels,Text_Tag,Length,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10
0,0,1.0,abortion,82,-0.843113,0.354266,0.356839,0.419612,0.502846,0.265881,1.124245,-0.337539,-1.011597,0.389831,-1.143914,-1.498768,0.349013,0.630598,-0.308929,-1.211497,0.639380,-0.070070,0.197310,-0.694955,-0.748948,-0.985393,-1.381263,-0.965011,0.196261,-0.165667,-0.192807,-0.399925,-0.571577,-0.015549,0.400864,-0.732438,0.259238,0.038886,-1.680591,-0.752563,...,0.552735,-0.799284,-0.250765,-1.899152,-0.518160,0.620628,0.464231,-0.309856,0.085772,0.445845,0.001798,0.197886,1.256550,-0.412361,-0.562702,-1.153980,0.355673,-0.240783,-1.033721,1.400332,0.022141,0.510811,0.853409,0.351012,0.043098,0.612751,-0.362309,0.728115,-0.631633,-0.265538,1.828789,-2.132058,1.057840,-4.186034,-0.508924,-7.662283,-3.929191,-4.275534,-1.816664,-1.815194
1,1,2.0,"energy,history,job-accomplishments",141,-0.154687,0.872684,1.772497,-0.295066,0.222306,-0.253726,-1.300731,0.190504,0.343304,-0.872295,0.883556,-0.409843,-0.357488,-0.139657,0.384983,-0.794849,-1.066065,-0.051639,0.139327,0.139749,0.117898,0.461543,-1.046779,0.199070,-1.169691,-0.055779,-0.828227,-1.178741,0.154195,-1.155196,0.319821,-0.137026,-0.893601,-0.493398,0.772720,-0.778966,...,1.213236,-0.706025,-0.896191,-0.587146,0.188100,0.173347,-1.696494,-0.575991,-0.401825,-0.878120,0.333844,-0.645788,-0.933316,0.095632,-1.189360,-0.747389,-0.634917,1.291249,-1.595213,0.180152,-0.493622,0.729952,-0.513198,1.092363,-0.258281,0.267083,-0.399990,-0.463839,-1.466221,0.220743,4.032458,0.975797,-0.608706,-6.131517,2.795102,5.009892,-2.136959,-0.229702,-1.771302,-4.286652
2,2,3.0,foreign-policy,105,-1.090439,-1.039243,-0.442930,-1.469323,0.165641,-0.411887,-0.524987,1.435755,-0.221388,0.336802,-0.551851,-0.579335,-1.103295,1.014058,0.514499,0.020389,1.099066,1.208101,0.988841,1.010967,-0.034147,-1.322206,-1.929260,-0.313306,-0.597575,0.154907,0.451024,-0.220749,-0.292568,-0.569620,-1.047028,-1.128314,0.382467,1.066755,-0.906427,-0.978917,...,0.285562,0.600908,-0.681900,1.709387,-0.631731,0.168582,-0.115423,0.145797,-1.153039,0.255021,-1.061557,0.622468,0.076881,-1.268889,0.564817,0.068600,-0.259890,0.401971,-0.329564,1.528328,0.896938,1.188235,-0.614642,0.053428,-0.562576,0.325780,-1.103372,-1.954507,0.481241,-0.146407,11.340319,-4.131258,6.469006,-4.635067,0.957096,2.865126,2.294436,-2.089983,-1.433335,-2.065531
3,3,1.0,health-care,78,0.599440,0.623602,0.633819,0.405363,0.706752,-0.079211,0.344765,-0.220469,-0.569685,-0.079065,-0.076758,1.569412,-0.278345,0.464309,-0.433065,-1.224822,-0.210659,-0.190351,-0.811026,-1.186909,-0.193128,0.878078,1.618985,-0.496415,1.310311,-0.229879,-0.483367,-0.503116,0.748487,-0.606732,-1.408139,-0.482079,-0.711370,0.238348,-0.895124,0.667424,...,0.494266,-0.434182,-1.080807,-0.713855,-0.633628,0.060904,0.127755,-0.921143,-0.846210,-0.678195,-0.489322,-1.591884,0.925410,-0.127750,0.404707,-0.285969,0.945852,-0.417142,-1.571512,1.861141,0.311482,0.002135,-0.285264,0.073709,0.787635,-0.344154,-0.042826,1.363384,0.595176,-0.336081,-0.317802,-2.836876,-1.256525,2.278089,7.042942,-6.476668,-8.591688,-0.741392,-6.286454,-0.008310
4,4,2.0,"economy,jobs",54,-0.092456,-0.040649,-0.215230,0.439570,1.114174,0.060738,-0.115374,-0.424731,0.706268,-0.155484,0.662839,0.791876,-0.067190,-0.313515,-0.166140,0.178111,-0.376365,0.253623,-1.166783,-1.275674,0.633949,0.408042,-1.205060,-1.429486,-0.148765,0.656819,-1.473106,0.556760,-0.075808,-1.473760,0.632215,1.001486,0.441519,0.425402,-0.661800,2.639184,...,2.571323,1.039394,-0.520890,-1.399412,1.457771,0.471627,-2.558429,-0.643707,-0.414140,0.643783,0.277404,-2.057265,-0.369222,0.343270,-0.080974,-1.999635,0.116802,-0.130946,-2.222349,0.629590,-0.951390,-0.129189,-0.908807,1.449807,0.192440,0.047139,-0.33

In [ ]:
train_df = full_df[full_df.Labels.notnull()]
test_df = full_df[full_df.Labels.isnull()]

In [ ]:
train_df

,index,Labels,Text_Tag,Length,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10
0,0,1.0,abortion,82,-0.843113,0.354266,0.356839,0.419612,0.502846,0.265881,1.124245,-0.337539,-1.011597,0.389831,-1.143914,-1.498768,0.349013,0.630598,-0.308929,-1.211497,0.639380,-0.070070,0.197310,-0.694955,-0.748948,-0.985393,-1.381263,-0.965011,0.196261,-0.165667,-0.192807,-0.399925,-0.571577,-0.015549,0.400864,-0.732438,0.259238,0.038886,-1.680591,-0.752563,...,0.552735,-0.799284,-0.250765,-1.899152,-0.518160,0.620628,0.464231,-0.309856,0.085772,0.445845,0.001798,0.197886,1.256550,-0.412361,-0.562702,-1.153980,0.355673,-0.240783,-1.033721,1.400332,0.022141,0.510811,0.853409,0.351012,0.043098,0.612751,-0.362309,0.728115,-0.631633,-0.265538,1.828789,-2.132058,1.057840,-4.186034,-0.508924,-7.662283,-3.929191,-4.275534,-1.816664,-1.815194
1,1,2.0,"energy,history,job-accomplishments",141,-0.154687,0.872684,1.772497,-0.295066,0.222306,-0.253726,-1.300731,0.190504,0.343304,-0.872295,0.883556,-0.409843,-0.357488,-0.139657,0.384983,-0.794849,-1.066065,-0.051639,0.139327,0.139749,0.117898,0.461543,-1.046779,0.199070,-1.169691,-0.055779,-0.828227,-1.178741,0.154195,-1.155196,0.319821,-0.137026,-0.893601,-0.493398,0.772720,-0.778966,...,1.213236,-0.706025,-0.896191,-0.587146,0.188100,0.173347,-1.696494,-0.575991,-0.401825,-0.878120,0.333844,-0.645788,-0.933316,0.095632,-1.189360,-0.747389,-0.634917,1.291249,-1.595213,0.180152,-0.493622,0.729952,-0.513198,1.092363,-0.258281,0.267083,-0.399990,-0.463839,-1.466221,0.220743,4.032458,0.975797,-0.608706,-6.131517,2.795102,5.009892,-2.136959,-0.229702,-1.771302,-4.286652
2,2,3.0,foreign-policy,105,-1.090439,-1.039243,-0.442930,-1.469323,0.165641,-0.411887,-0.524987,1.435755,-0.221388,0.336802,-0.551851,-0.579335,-1.103295,1.014058,0.514499,0.020389,1.099066,1.208101,0.988841,1.010967,-0.034147,-1.322206,-1.929260,-0.313306,-0.597575,0.154907,0.451024,-0.220749,-0.292568,-0.569620,-1.047028,-1.128314,0.382467,1.066755,-0.906427,-0.978917,...,0.285562,0.600908,-0.681900,1.709387,-0.631731,0.168582,-0.115423,0.145797,-1.153039,0.255021,-1.061557,0.622468,0.076881,-1.268889,0.564817,0.068600,-0.259890,0.401971,-0.329564,1.528328,0.896938,1.188235,-0.614642,0.053428,-0.562576,0.325780,-1.103372,-1.954507,0.481241,-0.146407,11.340319,-4.131258,6.469006,-4.635067,0.957096,2.865126,2.294436,-2.089983,-1.433335,-2.065531
3,3,1.0,health-care,78,0.599440,0.623602,0.633819,0.405363,0.706752,-0.079211,0.344765,-0.220469,-0.569685,-0.079065,-0.076758,1.569412,-0.278345,0.464309,-0.433065,-1.224822,-0.210659,-0.190351,-0.811026,-1.186909,-0.193128,0.878078,1.618985,-0.496415,1.310311,-0.229879,-0.483367,-0.503116,0.748487,-0.606732,-1.408139,-0.482079,-0.711370,0.238348,-0.895124,0.667424,...,0.494266,-0.434182,-1.080807,-0.713855,-0.633628,0.060904,0.127755,-0.921143,-0.846210,-0.678195,-0.489322,-1.591884,0.925410,-0.127750,0.404707,-0.285969,0.945852,-0.417142,-1.571512,1.861141,0.311482,0.002135,-0.285264,0.073709,0.787635,-0.344154,-0.042826,1.363384,0.595176,-0.336081,-0.317802,-2.836876,-1.256525,2.278089,7.042942,-6.476668,-8.591688,-0.741392,-6.286454,-0.008310
4,4,2.0,"economy,jobs",54,-0.092456,-0.040649,-0.215230,0.439570,1.114174,0.060738,-0.115374,-0.424731,0.706268,-0.155484,0.662839,0.791876,-0.067190,-0.313515,-0.166140,0.178111,-0.376365,0.253623,-1.166783,-1.275674,0.633949,0.408042,-1.205060,-1.429486,-0.148765,0.656819,-1.473106,0.556760,-0.075808,-1.473760,0.632215,1.001486,0.441519,0.425402,-0.661800,2.639184,...,2.571323,1.039394,-0.520890,-1.399412,1.457771,0.471627,-2.558429,-0.643707,-0.414140,0.643783,0.277404,-2.057265,-0.369222,0.343270,-0.080974,-1.999635,0.116802,-0.130946,-2.222349,0.629590,-0.951390,-0.129189,-0.908807,1.449807,0.192440,0.047139,-0.33

In [ ]:


X, y = train_df.drop(['index', 'Text_Tag', 'Labels'], axis=1), train_df.Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
model_cat = CatBoostClassifier(od_type='Iter', iterations=20000, task_type='GPU')
model_cat.fit(X_train, y_train.astype(int),
              eval_set=(X_test, y_test.astype(int)),
              early_stopping_rounds=100)

Learning rate set to 0.047543


0:	learn: 1.7862222	test: 1.7878028	best: 1.7878028 (0)	total: 63.7ms	remaining: 10m 37s
1:	learn: 1.7806778	test: 1.7837998	best: 1.7837998 (1)	total: 119ms	remaining: 9m 53s
2:	learn: 1.7752955	test: 1.7801251	best: 1.7801251 (2)	total: 187ms	remaining: 10m 22s
3:	learn: 1.7701871	test: 1.7768494	best: 1.7768494 (3)	total: 239ms	remaining: 9m 57s
4:	learn: 1.7653897	test: 1.7736957	best: 1.7736957 (4)	total: 293ms	remaining: 9m 46s
5:	learn: 1.7610774	test: 1.7704375	best: 1.7704375 (5)	total: 346ms	remaining: 9m 35s
6:	learn: 1.7573874	test: 1.7675637	best: 1.7675637 (6)	total: 391ms	remaining: 9m 17s
7:	learn: 1.7534072	test: 1.7651080	best: 1.7651080 (7)	total: 439ms	remaining: 9m 8s
8:	learn: 1.7491572	test: 1.7628027	best: 1.7628027 (8)	total: 505ms	remaining: 9m 20s
9:	learn: 1.7449932	test: 1.7600648	best: 1.7600648 (9)	total: 552ms	remaining: 9m 11s
10:	learn: 1.7414594	test: 1.7579116	best: 1.7579116 (10)	total: 599ms	remaining: 9m 3s
11:	learn: 1.7383381	test: 1.7559463	bes

In [ ]:
# FINDING THE IMPORTANT FEATURES

importance = pd.DataFrame(data=model_cat.feature_importances_, index=X_train.columns, columns=['imp']).sort_values(by='imp',ascending=False)
imp_feat = importance[importance['imp'] > 0.05].index

In [ ]:


# AFTER SELECTING THE IMPORTANT FEATURES

model_cat = CatBoostClassifier(od_type='Iter', iterations=10000, task_type='GPU')
model_cat.fit(X_train[imp_feat], y_train.astype(int),
              eval_set=(X_test[imp_feat], y_test.astype(int)),
              early_stopping_rounds=100)



Learning rate set to 0.047543


0:	learn: 1.7862828	test: 1.7880859	best: 1.7880859 (0)	total: 56.9ms	remaining: 9m 28s
1:	learn: 1.7811130	test: 1.7844510	best: 1.7844510 (1)	total: 93ms	remaining: 7m 44s
2:	learn: 1.7761745	test: 1.7806592	best: 1.7806592 (2)	total: 139ms	remaining: 7m 41s
3:	learn: 1.7709746	test: 1.7766843	best: 1.7766843 (3)	total: 176ms	remaining: 7m 18s
4:	learn: 1.7661058	test: 1.7732569	best: 1.7732569 (4)	total: 221ms	remaining: 7m 21s
5:	learn: 1.7618951	test: 1.7703390	best: 1.7703390 (5)	total: 264ms	remaining: 7m 19s
6:	learn: 1.7578626	test: 1.7680247	best: 1.7680247 (6)	total: 304ms	remaining: 7m 13s
7:	learn: 1.7535934	test: 1.7651900	best: 1.7651900 (7)	total: 340ms	remaining: 7m 4s
8:	learn: 1.7501881	test: 1.7628189	best: 1.7628189 (8)	total: 374ms	remaining: 6m 55s
9:	learn: 1.7464981	test: 1.7606807	best: 1.7606807 (9)	total: 409ms	remaining: 6m 48s
10:	learn: 1.7424359	test: 1.7584578	best: 1.7584578 (10)	total: 453ms	remaining: 6m 51s
11:	learn: 1.7393208	test: 1.7563072	best:

In [ ]:


prediction = model_cat.predict_proba(test_df[X_train.columns])
submission = pd.DataFrame(prediction, columns=[0,1,2,3,4,5])
submission.to_csv('submission.csv', index=False)
submission



,0,1,2,3,4,5
0,0.142919,0.184484,0.214694,0.229621,0.069309,0.158973
1,0.130247,0.241526,0.185295,0.285460,0.043765,0.113708
2,0.150753,0.272912,0.126161,0.176447,0.079953,0.193774
3,0.252152,0.250516,0.187022,0.119719,0.104570,0.086021
4,0.109355,0.306551,0.190751,0.152311,0.142079,0.098953
...,...,...,...,...,...,...
1262,0.116174,0.125111,0.320891,0.280894,0.021846,0.135085
1263,0.123580,0.149011,0.107320,0.240439,0.061240,0.318410
1264,0.234699,0.236047,0.195547,0.100265,0.121422,0.112020
1265,0.150347,0.169468,0.209216,0.230967,0.074762,0.165240


In [ ]:


prediction = model_cat.predict_proba(test_df[X_train.columns])
submission = pd.DataFrame(prediction, columns=[0,1,2,3,4,5])
submission.to_csv('submission.csv', index=False)

